In [ ]:
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.metrics import mean_absolute_error
import json

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
dataset=pd.read_csv('eplusout_3years.csv')

with open('opt_settings.JSON', 'r') as f:
        datastore = json.load(f)
        
opt_units=datastore['opt_units']
opt_e=datastore['opt_e']
opt_bs=datastore['opt_bs']
opt_stamps=datastore['opt_stamps']

In [ ]:
# load dataset
values=dataset.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# specify the number of features
n_features = len(list(dataset.columns)) #number of columns in dataset
n_output = 18 # 3hrs
n_train_stamps = 2*364*24*6 #2 years
# list to store the results
results=[]

In [ ]:
def Reframing(n_stamps, n_output=n_output, scaled=scaled, n_train_stamps = n_train_stamps):

  # frame as supervised learning
    reframed = series_to_supervised(scaled, n_stamps, n_output)

    # split into train and test sets
    values = reframed.values
    train = values[:n_train_stamps, :]
    test = values[n_train_stamps:, :]

    # split into input and outputs
    n_obs = n_stamps * n_features
    train_X, train_y = train[:, :n_obs], train[:, -n_features]
    test_X, test_y = test[:, :n_obs], test[:, -n_features]
    
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], n_stamps, n_features))
    test_X = test_X.reshape((test_X.shape[0], n_stamps, n_features))
    
    return train_X, train_y, test_X, test_y

In [ ]:
def PlotResult(d, case, dataset=dataset, n_train_stamps = n_train_stamps):

  filename=f"n_stamps{d['n_stamps']}_epochs{d['epochs']}_batchsize{d['batch_size']}_n_units{d['n_units']}"
  history=d['history']
  inv_yhat=d['inv_yhat']

  # plot history
  fig0 = pyplot.figure()
  pyplot.plot(history.history['loss'], label='train')
  pyplot.plot(history.history['val_loss'], label='test')
  pyplot.xlabel('Epoch')
  pyplot.ylabel('Loss (MAE)')
  pyplot.title('Loss (MAE) for '+ case)
  pyplot.legend()
  fig0.savefig(filename+'_loss.png', dpi=120)
  pyplot.show()

  # plot prediction
  fig1 = pyplot.figure()
  pyplot.plot(dataset['avg_T_in'][n_train_stamps:].values, label='Real')
  pyplot.plot(inv_yhat, label='Predicted')
  pyplot.legend()
  pyplot.xlabel('Timestamp')
  pyplot.ylabel('Tin (°C)')
  pyplot.title("Real Tin vs. Predicted Tin for "+ case)
  fig1.savefig(filename+'_results.png', dpi=120)
  print(fig1.dpi)
  pyplot.show()

In [ ]:
def ChangeModel(n_units=50, e=10, bs=72, n_stamps=6, n_output=n_output, scaled=scaled, to_reframe=False, *args):
  
  if to_reframe:
    train_X, train_y, test_X, test_y = Reframing(n_stamps)
    print('reframed')
    
  else:
    train_X = args[0]
    train_y = args[1]
    test_X = args[2]
    test_y = args[3]


  # design network
  model = Sequential()
  model.add(LSTM(n_units, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(Dense(1))
  model.compile(loss='mae', optimizer='adam')

  # fit network
  history = model.fit(train_X, train_y, epochs=e, batch_size=bs, validation_data=(test_X, test_y), verbose=2, shuffle=False)
  


  # make a prediction
  yhat = model.predict(test_X)
  test_X = test_X.reshape((test_X.shape[0], n_stamps*n_features))
  
  # invert scaling for forecast
  inv_yhat = concatenate((yhat, test_X[:, -(n_features-1):]), axis=1)
  inv_yhat = scaler.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  
  # invert scaling for actual
  test_y = test_y.reshape((len(test_y), 1))
  inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
  inv_y = scaler.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  
  # calculate RMSE and MAE
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  mae = mean_absolute_error(inv_y, inv_yhat)
  d={'n_units': n_units, 'epochs': e, 'batch_size':bs, 'n_stamps': n_stamps, 'RMSE': rmse, 'MAE': mae, "history": history, "inv_yhat": inv_yhat}
  print('\n', d)
  #print('Test RMSE: %.3f' % rmse)

  return d

In [ ]:
# first choice for parameters
train_X, train_y, test_X, test_y = Reframing(1)
results.append(ChangeModel(n_units=50, e=10, bs=72, n_stamps=1, n_output=n_output, scaled=scaled, to_reframe=False, train_X, train_y, test_X, test_y))

In [ ]:
# optimal parameters
train_X, train_y, test_X, test_y = Reframing(opt_stamps)
results.append(ChangeModel(n_units=opt_units, opt_e=10, opt_bs=72, n_stamps=opt_stamps, n_output=n_output, scaled=scaled, to_reframe=False, train_X, train_y, test_X, test_y))

In [ ]:
# predict value 24hrs later
train_X, train_y, test_X, test_y = Reframing(opt_stamps,n_output=144)
results.append(ChangeModel(n_units=opt_units, opt_e=10, opt_bs=72, n_stamps=opt_stamps, n_output=144, scaled=scaled, to_reframe=False, train_X, train_y, test_X, test_y))

In [ ]:
# predict value 1 week later
train_X, train_y, test_X, test_y = Reframing(opt_stamps,n_output=1008)
results.append(ChangeModel(n_units=opt_units, opt_e=10, opt_bs=72, n_stamps=opt_stamps, n_output=1008, scaled=scaled, to_reframe=False, train_X, train_y, test_X, test_y))